In [1]:
import boto3
import os
import json
import logging
import scrapy
import requests

import pandas as pd

from datetime import datetime, timedelta
from scrapy.crawler import CrawlerProcess
from dotenv import load_dotenv
load_dotenv()
%load_ext dotenv
%dotenv

## Recuperation des données météo

In [2]:
villes = ["Le Mont-Saint-Michel","Saint-Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Château du Haut-Kœnigsbourg","Colmar","Eguisheim","Besançon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes-les-Mimosas","Cassis","Marseille","Aix-en-Provence","Avignon","Uzès","Nîmes","Aigues-Mortes","Les Saintes-Maries-de-la-Mer","Collioure","Carcassonne","Ariège","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]
key = os.getenv("APIKEY")

In [3]:
temps = []
for ville in villes:
    url = f"https://nominatim.openstreetmap.org/?addressdetails=1&q={ville}&format=json&limit=1"
    r = requests.get(url)
    lat = r.json()[0]["lat"]
    lon = r.json()[0]["lon"]
    a = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={key}&units=metric")
    data = a.json()
    temps.append({"ville_rechercher" : ville, data.get('list')[1].get('dt_txt'): {"temperature" : data.get('list')[1].get('main').get('temp_max'),},
                                    data.get('list')[9].get('dt_txt'): {"temperature" :data.get('list')[9].get('main').get('temp_max')},
                                    data.get('list')[17].get('dt_txt'): {"temperature" :data.get('list')[17].get('main').get('temp_max')},
                                    data.get('list')[25].get('dt_txt'): {"temperature" :data.get('list')[25].get('main').get('temp_max')},
                                    data.get('list')[33].get('dt_txt'): {"temperature" :data.get('list')[33].get('main').get('temp_max')}}) 



In [4]:
with open('temps.json', 'w', newline="") as f:
        json.dump(temps, f) 
        

In [5]:
class QuotesSpider1(scrapy.Spider):
    name = "spider1"
    start_urls = ['https://www.booking.com/index.fr.html']
    
    def parse(self, response):
        villes = ["Le Mont-Saint-Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon", 
                "Bormes les Mimosas","Cassis","Marseille","Aix-en-Provence","Avignon","Uzès","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]
        for x in villes:
            yield scrapy.FormRequest.from_response(
            response,
            formdata={'ss': x },
            callback=self.after_search
        )
    def after_search(self, response):
        villes = response.url.split("ss=")[-1].split("&")[0]
        keys = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.ef9845d4b3.da89aeb942')
        url_hotels = [] 

        for key in keys:
            recherche = response.css('input.ce45093752::attr(value)').get()
            url_hotel = key.css('a.e13098a59f::attr(href)').get()
            url_hotels.append(url_hotel)
            
            yield {
                "ville_rechercher" : recherche,
                "ville_précise": key.css("span.f4bd0794db.b4273d69aa::text").get(),
                "hotel": key.css('div.fcab3ed991.a23c043802::text').get(),
                "url": key.css('a.e13098a59f::attr(href)').get(),
                "note": key.css('div.b5cd09854e.d10a6220b4::text').get() 
            }

        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)
                            
filename = "hotel_1.json"

if filename in os.listdir():
        os.remove( filename)
        
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

process.crawl(QuotesSpider1)
process.start()

2022-11-22 20:43:57 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-11-22 20:43:57 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.22621-SP0
2022-11-22 20:43:57 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True, 'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-11-22 20:43:57 [py.warnings] WARNING: c:\Users\jerem\AppData\Local\Programs\Python\Python310\lib\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for bac

In [2]:
class QuotesSpider2(scrapy.Spider):
    name = "spider2"
    file = open("hotel_1.json")
    file = json.load(file)
    liste_urls = [element["url"] for element in file] 

    start_urls = liste_urls
        
    def parse(self, response):
            
            a = response.css('#hotel_address::attr(data-atlas-latlng)').get(),
            b = a[0]
            c = b.split(",")
            
            yield {
                "hotel": response.css('h2.pp-header__title::text').get(),
                "description" : response.css('#property_description_content > p::text').get(),
                "latitutde": c[0],
                "longitude": c[1],
            }
        
filename = "hotel_2.json"

if filename in os.listdir():
        os.remove( filename)
        
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

process.crawl(QuotesSpider2)
process.start()

2022-11-22 20:45:56 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-11-22 20:45:56 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.2 (tags/v3.10.2:a58ebcc, Jan 17 2022, 14:12:15) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.22621-SP0
2022-11-22 20:45:56 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True, 'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-11-22 20:45:56 [py.warnings] WARNING: c:\Users\jerem\AppData\Local\Programs\Python\Python310\lib\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for bac

In [3]:
df_1 = pd.read_json("hotel_1.json")
df_2 = pd.read_json("hotel_2.json")
df_3 = pd.read_json("temps.json")

In [8]:
df_3

,ville_rechercher,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00
0,Le Mont-Saint-Michel,"{'temperature': 8.29, 'temps': 'overcast clouds'}","{'temperature': 9.5, 'temps': 'light rain'}","{'temperature': 9.26, 'temps': 'scattered clou...","{'temperature': 8.38, 'temps': 'broken clouds'}","{'temperature': 10.16, 'temps': 'light rain'}"
1,Saint-Malo,"{'temperature': 10.01, 'temps': 'scattered clo...","{'temperature': 10.19, 'temps': 'light rain'}","{'temperature': 10.11, 'temps': 'light rain'}","{'temperature': 9.36, 'temps': 'broken clouds'}","{'temperature': 11.94, 'temps': 'light rain'}"
2,Bayeux,"{'temperature': 7.49, 'temps': 'overcast clouds'}","{'temperature': 9.51, 'temps': 'light rain'}","{'temperature': 8.98, 'temps': 'light rain'}","{'temperature': 7.16, 'temps': 'broken clouds'}","{'temperature': 9.56, 'temps': 'light rain'}"
3,Le Havre,"{'temperature': 10.13, 'temps': 'light rain'}","{'temperature': 9.61, 'temps': 'light rain'}","{'temperature': 10.06, 'temps': 'overcast clou...","{'temperature': 10.69, 'temps': 'light rain'}","{'temperature': 9.92, 'temps': 'light rain'}"
4,Rouen,"{'temperature': 8.4, 'temps': 'broken clouds'}","{'temperature': 8.56, 'temps': 'broken clouds'}","{'temperature': 7.21, 'temps': 'broken clouds'}","{'temperature': 6.57, 'temps': 'scattered clou...","{'temperature': 7.86, 'temps': 'overcast clouds'}"
5,Paris,"{'temperature': 8.87, 'temps': 'scattered clou...","{'temperature': 8.9, 'temps': 'scattered clouds'}","{'temperature': 9.66, 'temps': 'light rain'}","{'temperature': 7.65, 'temps': 'few clouds'}","{'temperature': 8.24, 'temps': 'overcast clouds'}"
6,Amiens,"{'temperature': 8.56, 'temps': 'overcast clouds'}","{'temperature': 8.66, 'temps': 'broken clouds'}","{'temperature': 7.85, 'temps': 'light rain'}","{'temperature': 6.52, 'temps': 'scattered clou...","{'temperature': 7.5, 'temps': 'overcast clouds'}"
7,Lille,"{'temperature': 8.76, 'temps': 'light rain'}","{'temperature': 9.12, 'temps': 'light rain'}","{'temperature': 8.39, 'temps': 'light rain'}","{'temperature': 7.26, 'temps': 'broken clouds'}","{'temperature': 7.62, 'temps': 'overcast clouds'}"
8,Strasbourg,"{'temperature': 6.7, 'temps': 'scattered clouds'}","{'temperature': 6.8100000000000005, 'temps': '...","{'temperature': 6.07, 'temps': 'overcast clouds'}","{'temperature': 8.4, 'temps': 'overcast clouds'}","{'temperature': 4.41, 'temps': 'few clouds'}"
9,Château du Haut-Kœnigsbourg,"{'temperature': 3.56, 'temps': 'broken clouds'}","{'temperature': 4.25, 'temps': 'broken clouds'}","{'temperature': 3.77, 'temps': 'overcast clouds'}","{'temperature': 4.8, 'temps': 'overcast clouds'}","{'temperature': 1.15, 'temps': 'few clouds'}"


In [4]:
df_final = pd.merge(df_1, df_2, how='outer', on='hotel')

In [6]:
df_final = pd.merge(df_final, df_3, how='outer', on='ville_rechercher')

In [7]:
df_final

,ville_rechercher,ville_précise,hotel,url,note,description,latitutde,longitude,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00
0,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0",Vous pouvez bénéficier d'une réduction Genius ...,48.614700,-1.509617,"{'temperature': 8.29, 'temps': 'overcast clouds'}","{'temperature': 9.5, 'temps': 'light rain'}","{'temperature': 9.26, 'temps': 'scattered clou...","{'temperature': 8.38, 'temps': 'broken clouds'}","{'temperature': 10.16, 'temps': 'light rain'}"
1,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2",Installé dans des espaces verts à seulement 2 ...,48.614247,-1.510545,"{'temperature': 8.29, 'temps': 'overcast clouds'}","{'temperature': 9.5, 'temps': 'light rain'}","{'temperature': 9.26, 'temps': 'scattered clou...","{'temperature': 8.38, 'temps': 'broken clouds'}","{'temperature': 10.16, 'temps': 'light rain'}"
2,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1",L'hôtel De La Digue est un établissement tradi...,48.616882,-1.510918,"{'temperature': 8.29, 'temps': 'overcast clouds'}","{'temperature': 9.5, 'temps': 'light rain'}","{'temperature': 9.26, 'temps': 'scattered clou...","{'temperature': 8.38, 'temps': 'broken clouds'}","{'temperature': 10.16, 'temps': 'light rain'}"
3,Le Mont-Saint-Michel,Le Mont-Saint-Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,0",Vous pouvez bénéficier d'une réduction Genius ...,48.635085,-1.510540,"{'temperature': 8.29, 'temps': 'overcast clouds'}","{'temperature': 9.5, 'temps': 'light rain'}","{'temperature': 9.26, 'temps': 'scattered clou...","{'temperature': 8.38, 'temps': 'broken clouds'}","{'temperature': 10.16, 'temps': 'light rain'}"
4,Le Mont-Saint-Michel,Le Mont-Saint-Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3",Vous pouvez bénéficier d'une réduction Genius ...,48.635349,-1.510379,"{'temperature': 8.29, 'temps': 'overcast clouds'}","{'temperature': 9.5, 'temps': 'light rain'}","{'temperature': 9.26, 'temps': 'scattered clou...","{'temperature': 8.38, 'temps': 'broken clouds'}","{'temperature': 10.16, 'temps': 'light rain'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,Lyon,"5e arr., Lyon",Quadraverde,https://www.booking.com/hotel/fr/residence-rao...,"7,8",Le Quadraverde propose des hébergements indépe...,45.756093,4.789343,"{'temperature': 5.78, 'temps': 'few clouds'}","{'temperature': 5.83, 'temps': 'clear sky'}","{'temperature': 8.74, 'temps': 'overcast clouds'}","{'temperature': 7.22, 'temps': 'scattered clou...","{'temperature': 5.89, 'temps': 'overcast clouds'}"
871,Lyon,"2e arr., Lyon",Citadines Presqu'île Lyon,https://www.booking.com/hotel/fr/citadines-apa...,"8,3",Le Citadines Presqu'île Lyon est situé dans le...,45.761587,4.832627,"{'temperature': 5.78, 'temps': 'few clouds'}","{'temperature': 5.83, 'temps': 'clear sky'}","{'temperature': 8.74, 'temps': 'overcast clouds'}","{'temperature': 7.22, 'temps': 'scattered clou...","{'temperature': 5.89, 'temps': 'overcast clouds'}"
872,Lyon,"5e arr., Lyon",MiHotel Tour Rose,https://www.booking.com/hotel/fr/mihotel-tour-...,"8,5",Vous pouvez bénéficier d'une réduction Genius ...,45.762725,4.826820,"{'temperature': 5.78, 'temps': 'few clouds'}","{'temperature': 5.83, 'temps': 'clear sky'}","{'temperature': 8.74, 'temps': 'overcast clouds'}","{'temperature': 7.22, 'temps': 'scattered clou...","{'temperature': 5.89, 'temps': 'overcast clouds'}"
873,Lyon,"2e arr., Lyon",Charming 1br at the heart of Lyon 5 min walkin...,https://www.booking.com/hotel/fr/charming-1br-...,"8,3","Situé dans le centre de Lyon, à moins de 1 km ...",45.755370,4.829876,"{'temperature': 5.78, 'temps': 'few clouds'}","{'temperature': 5.83, 'temps': 'clear sky'}","{'temperature': 8.74, 'temps': 'o

In [ ]:
access_key = os.getenv("ACCESS_KEY")
access_key_secrete = os.getenv("SECRET_ACCESS_KEY")
buket_name = os.getenv("BUCKETNAME")

session = boto3.Session(aws_access_key_id=access_key, 
                        aws_secret_access_key=access_key_secrete)
s3 = session.resource("s3")
bucket = s3.create_bucket(Bucket=buket_name, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})
put_object = bucket.put_object(Key="hotel_1.csv", Body=df_1)